In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        deposits(where: {timestamp_lte: """ + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            user {
                id
            }
            caller {
                id
            }
            reserve {
                symbol
            }
            amount
            timestamp
        }
    }
    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})
    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["deposits"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue
    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["user"] = transaction["user"]["id"]
        transaction["reserve"] = transaction["reserve"]["symbol"]
        transaction["caller"] = transaction["caller"]["id"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1

with open('deposit.csv', 'w', newline='') as output_file:
    DICT_WRITER = csv.DictWriter(output_file, keys)
    DICT_WRITER.writeheader()
    DICT_WRITER.writerows(transactions)
        

1000 transactions found at timestamp 1654016400
999 transactions found at timestamp 1653785199
999 transactions found at timestamp 1653587251
999 transactions found at timestamp 1653359605
999 transactions found at timestamp 1653134734
999 transactions found at timestamp 1652951323
999 transactions found at timestamp 1652784536
999 transactions found at timestamp 1652641061
999 transactions found at timestamp 1652517929
999 transactions found at timestamp 1652387005
999 transactions found at timestamp 1652331183
999 transactions found at timestamp 1652269459
999 transactions found at timestamp 1652142473
999 transactions found at timestamp 1652083130
999 transactions found at timestamp 1652019914
999 transactions found at timestamp 1651998813
999 transactions found at timestamp 1651973828
996 transactions found at timestamp 1651925923
996 transactions found at timestamp 1651907749
999 transactions found at timestamp 1651859052
999 transactions found at timestamp 1651750570
999 transact

In [2]:
!wc -l deposit.csv
# 290781

294139 deposit.csv


In [3]:
from google.colab import files
files.download('deposit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>